In [ ]:
using Random, Plots, Statistics, PyCall
gr()
pydisp = pyimport("IPython.lib.display")

# IL027: Interdisciplinary Computer Modelling

## Image Processing

#### Laura Cooper, Warwick Mathematics Institute

## Introduction

<img src="img/L8/marvel.jpg" alt="marvel" style="width: 400px;" align="right" />

<br><br>
Image processing is a part of every day life, whether applying a filter on instagram or snapchat, watching a film or seeing an advert. The term "to photoshop" is even being used to describe an altered photo, after the most widely used image processing software.


<img src="img/L8/ej.jpg" alt="ej" style="width: 400px;" align="right" />

<br><br>
Image processing is used in both art and science. Photo manipulation is an art form where the artist creates an illusion by changing and combining photographs. This is an example from Erik Johansson.

Have a look at [Erik Johansson's Youtube channel](https://www.youtube.com/user/tackochgodnatt) to see how he creates his artwork.

In [ ]:
using Images, TestImages, FFTW

# Images : a collection of packages focused on image processing.
# TestImages : a standard collection of test images. 
# FFTW: a library for fast Fourier transforms (FFTs), as well as tools useful for signal processing.

## Example of Image Processing in Action: Spot the Root

In science image processsing is used to analysis image data in many areas. One example is looking at roots in soil. Roots can be visualised in soil using X-Ray Computed Tomorgraphy (XCT). This [video shows how XCT works](https://www.youtube.com/watch?v=o47ua5joJto&t=20s).

In [ ]:
pydisp.YouTubeVideo("o47ua5joJto", start=20)

See if you can spot the roots in the images below, before loading the image with the roots highlighted 

In [ ]:
#Load image without highlight
load("img/L8/Root1p2um.png")

In [ ]:
#Load image with highlight
load("img/L8/Root1p2umHighlight.png")

In [ ]:
#Load image without highlight
load("img/L8/Root50um.png")

In [ ]:
#Load image with highlight
load("img/L8/Root50umHighlight.png")

In [ ]:
#Load image without highlight
load("img/L8/Root60um.png")

In [ ]:
#Load image with highlight
load("img/L8/Root60umHighlight.png")

Visit [µ-VIS X-Ray Imaging Centre Southampton](https://www.youtube.com/channel/UCGgUXDYKG00QifY4JTq1x0w) for videos of various objects that have been in the XCT scanner.

# Description of greyscale/colour images as arrays

In [ ]:
man=testimage("mandrill") #Load an image

In [ ]:
manG=Gray.(man) #Convert to gray scale 

In [ ]:
#Show information about images
@show summary(man); #Summary of RGB image
@show summary(manG); #Summary of Gray image

The images are arrays of type N0f8:

> Normalized with 8 fractional bits, with 0 bits left for representing values higher than 1. Internally, these are represented in terms of the 8-bit unsigned integer UInt8

8-bit unsigned integer have ``2^8=256`` values which are scaled between the values of 0 and 1. For the colour image there are three channels, one each for red, green and blue, and combine to make the full colour image. For the gray image there is one channel. The value relates to the intensity of the pixel with 0 being black and 1 being white.

We can look at the values of individual pixels by calling them in the same way as entries in an array. A pixel from colour image has 3 values for red, green and blue. A pixel from the gray image has just one value.

In [ ]:
@show man[1,1]; #First pixel of RGB image
@show manG[1,1]; #First pixel of Gray image

Converting to grayscale is not as simple as calculating the mean of the red, green and blue channels. The values of each colour channel are weighted so that the luminance of the image is preserved.
You can read more here: https://en.wikipedia.org/wiki/Grayscale and https://juliaimages.org/latest/arrays_colors/

In [ ]:
#Take the mean of RGB to compare to the grayscale conversion
manC=channelview(man) #Separate image into red, green and blue colour channels
manA=sum.(manC[1,:,:].+manC[2,:,:].+manC[1,:,:])./3
plot(
    plot(manG,axis=false),
    plot(heatmap(manA,yflip=true,fill=(true,cgrad(:grays))),axis=false,cbar=false),layout=grid(1,2),size=(800,400)
)


The values of the pixels can be displayed as a histogram. This is the result for the gray scale image

In [ ]:
x,counts = imhist(manG,256); #Get the number of pixels (counts) for each grey value (x)
p1 = plot(x, counts, line=:stem,xlabel="Intensity",ylabel="Frequency",legend=false) #Plot histogram

The histogram shows the number of pixels in the image for each shade of gray. There are 256 shades between the values of 0 and 1.

We can also view the three channels of the colour image separately. The ``channelview`` command separates the three channels into three 2D arrays and outputs them as a 3 dimensional array.

In [ ]:
man_channels=channelview(man); #Separate colour image into three separate colour channels
@show summary(man_channels);

Displaying each of the channels as a separate gray scale image shows the red nose of the mandrill has high intensity pixels in the red channel where as the blue areas of the face have high intensity pixels shown in the blue channel.

In [ ]:
#Convert channels to grayscale to compare intensities
p1 = Gray.(man_channels[1,:,:]) #Red
p2 = Gray.(man_channels[2,:,:]) #Green
p3 = Gray.(man_channels[3,:,:]) #Blue
plot(
    plot(Gray.(man_channels[1,:,:]),axis=false,title="red"),
    plot(Gray.(man_channels[2,:,:]),axis=false,title="green"),
    plot(Gray.(man_channels[3,:,:]),axis=false,title="blue"),
    layout=grid(1,3),size=(900,300))

From here on we will only work with grayscale images, however the techniques can be adapted to colour images.

# Histograms

Histograms are one way we can visualise the information in an image. We can manipulate images to change the histogram, this can improve the appearance of the image and increase the contrast between different aspects of the image.

Equalising the histogram aims to make all intensities equally probable. This increase the contrast of the image to the human eye.

In [ ]:
#Equalise Histogram
hist_equalised_manG = histeq(manG, 256);
x_eq,counts_eq = imhist(hist_equalised_manG,256);
l = @layout (2,1)
p1 = plot(x, counts, xlims = (0,1), line=:stem ,ylabel="Frequency",legend=false)
p2 = plot(x_eq, counts_eq, xlims = (0,1), line=:stem,xlabel="Intensity",ylabel="Frequency",legend=false)
plot(p1, p2, layout = l)

In [ ]:
hist_equalised_manG #Display image with equalised histogram

Normalising a histogram scales the range of intensities to cover all 256 values using the maximum and minimum values of the original image.

In [ ]:
#Normalise Histogram
function normHist(GrayImage)
    minim=minimum(GrayImage) #Find minimum gray scale value of image
    maxim=maximum(GrayImage) #Find maximum gray scale value of image
    normalised=(GrayImage.-minim)./(maxim-minim) #Normalise the image
end

In [ ]:
N=normHist(manG) #Normalise histogram of image
Gray.(N) #Display resulting image

In [ ]:
#Compare the two histograms
x_n,counts_n = imhist(N,256);
l = @layout (2,1)
p1 = plot(x, counts, xlims = (0,1), line=:stem,ylabel="Frequency",legend=false);
p2 = plot(x_n, counts_n, xlims = (0,1), line=:stem,xlabel="Intensity",ylabel="Frequency",legend=false);
plot(p1, p2, layout = l)

This can make it easier to perform certain operations of the image and ensures that the full range of intensities is used.

# Thresholding

In image processing segmentation is used to divide an image into different areas of interest. This can be used for combining differet images together or for analysing the size and shape of objects. 

Thresholding is one way to segment an image. The image is changed to a binary image where all the pixels take either the value true (1) or false (0) depending on the threshold.

In [ ]:
thresh_manG=manG .> 0.6; #apply a threshold between the two peaks in the original histogram
Gray.(thresh_manG)

In [ ]:
#Show image as binary array
thresh_manG

By multiplying the grayscale image with the thresholded image parts of the image are removed.

In [ ]:
manG_T=manG.*thresh_manG

In [ ]:
#Plot histogram of manG_T
x_t,counts_t = imhist(manG_T, 256);
plot(x_t, counts_t, xlims = (0,1), ylims=(0,6000), line=:stem)

All pixels below the threshold of 0.6 have been assigned to 0. We could then go on to analyse the data contained in just the upper part of the histogram.

For further reading look up optimal thresholding, e.g. Otsu's method, which tries to find the optimal threshold value that separates an object from the background. 

# Filters

Applying filters to an image can help to remove noise and blur an image. The ``mapwindow`` function applies a function over a window of a defined size. For example, applying a median filter on a 3x3 window means each pixel takes the median value from the surrounding 9 pixels.

In [ ]:
lh=testimage("lighthouse") #load image

In [ ]:
lhG=Gray.(lh) #convert to grayscale

In [ ]:
mapwindow(median, lhG, (3,3)) # apply a median function on a window of size 3x3

In [ ]:
mapwindow(median, lhG, (31,31)) # apply a median function of a window of size 31x31

In [ ]:
mapwindow(mean, lhG, (31,31)) # apply a mean function of a window of size 31x31

In [ ]:
mapwindow(maximum, lhG, (31,31)) # apply a maximum function of a window of size 31x31

One issue with applying filters is what happens at the borders. There are four options:

>  * replicate (repeat edge values to infinity)
>  * circular (image edges "wrap around")
>  * symmetric (the image reflects relative to a position between pixels)
>  * reflect (the image reflects relative to the edge itself)

The default option is ``replicate``.

A Gaussian filter can be used to blur an image to remove noise and details. In 1D a Gaussian function looks like a bell curve. It is similar to applying a median filter but rather than giving all the pixels in the window the same weight, or importance, when calculating the average, the Gaussian filter gives more weight to the pixels closest to the centre of the window.

In [ ]:
Fs = 100;           # Sampling frequency
t = -0.5:1/Fs:0.5;  # Time vector 
L = length(t);      # Signal length
Gauss = 1/(4*sqrt(2*pi*0.1))*(exp.(-t.^2/(2*0.1^2))); #Gaussian signal assuming mean at 0 and standard deviation 0.1
plot(t,Gauss,xlabel="x",ylabel="G(x)") # 1D Gaussian

In [ ]:
lhG_G=imfilter(lhG, Kernel.gaussian(5)) #apply gaussian filter to image

The Gaussian filter has the effect of 'smoothing' the histogram. The Gaussian filter is an example of a low pass filter (further explaination can be found [here](https://homepages.inf.ed.ac.uk/rbf/HIPR2/gsmooth.htm))  The effect is to remove high spatial frequency components from the image. 

In [ ]:
x, counts=imhist(lhG);
x_g, counts_g=imhist(lhG_G);
l = @layout (2,1)
p1 = plot(x, counts, xlims = (0,1), ylims = (0,2.4e4), line=:stem, ylabel="Freqency",legend=false)
p2 = plot(x_g, counts_g, xlims = (0,1), ylims = (0,2.4e4), line=:stem, xlabel="Intensity",ylabel="Freqency",legend=false)
plot(p1, p2, layout = l)

## Sharpen Image
Most basic image processing software offers an option to sharpen an image. To do this we create a kernel by subtracting a "blurring" kernel. We make use of ```cld(x, y)``` which returns the smallest integer larger than or equal to x/y.

In [ ]:
function sharpenK(Kernel)
m,n=size(Kernel)
sharp=zeros(m,n) #Create zero array
sharp[cld(m,2),cld(n,2)]=sharp[cld(m,2),cld(n,2)]+2 #Add one to central pixel, 
sharp=sharp.-collect(Kernel) #Subtract the Gaussian (smoothing) kernel, collect replaces OffsetArray with Array
    return sharp
end

In [ ]:
sharpenK(Kernel.gaussian(1))

In [ ]:
lhG_S=imfilter(lhG, sharpenK(Kernel.gaussian(5))) #apply gaussian filter to image

In [ ]:
x_g, counts_g=imhist(lhG_S);
l = @layout (2,1)
p1 = plot(x, counts, xlims = (0,1), ylims = (0,2.4e4), line=:stem, ylabel="Freqency",legend=false)
p2 = plot(x_g, counts_g, xlims = (0,1), ylims = (0,2.4e4), line=:stem, xlabel="Intensity",ylabel="Freqency",legend=false)
plot(p1, p2, layout = l)

# Fourier Transform

Fourier Transform maps a signal to its component freqencies. Think about sound: Bass sounds (low pitch) come from low-freqency components and Treble sounds (high pitch) come from high-frequency components
* Remove hissing noise on a recording by removing the high frequecies
* Make next door party music by removing all high frequecies

We can create a signal, you can imagine its a recording picked up by a microphone. Adding some noise to the signal can be thought of as background sounds that were picked up during the recording.

In [ ]:
# Create a signal with some noise
Fs=1000; #Sample freqency
L=1500 # Length of signal
t=(0:L-1)/Fs # Time Vector
Hz1=20; #Signal 1
Hz2=80; #Signal 2
f=sin.(2*pi*Hz1*t)+sin.(2*pi*Hz2*t)+randn(Float64, size(t))
plot(1000*t,f, xlabel="Time", ylabel="Signal")

It is not possible to see the two signals in the above plot. Applying a Fourier transform to this signal shows the two signals clearly.

In [ ]:
Y=fft(f); # Fourier transform of function
P2=abs.(Y/L) #2 sided spectrum
P1=P2[1:(750+1)]; #1 sided spectrum
P1[2:end-1]=2*P1[2:end-1];
fd=Fs*(0:(L/2))/L; #Freqency fomain
l=@layout (2,1)
p1=plot(fd, P1)
p2=plot(fd, P1, xlims = (0,100),xlabel="Frequency")
plot(p1, p2, layout = l)

Let's try this with an image

In [ ]:
lena=testimage("lena_gray_256"); #Open an image
lenaG=Gray.(lena)#+0.5*Noise
rows, cols=size(lenaG)
#Add some noise bands to the image
x=0:rows-1
Noise=cos.(pi*x/16).*(ones(rows,cols))
im_noise=Float64.(lenaG)+0.5*Noise
Gray.(im_noise)

We can see how adding this noise effects the histogram of the image

In [ ]:
x_n, counts_n=imhist(im_noise);
plot(x_n, counts_n, xlims = (0,1), line=:stem, xlabel="Intensity",ylabel="Freqency",legend=false)

It is not possible to see what's noise and what's the original image from the histogram. We can apply a Fourier transform to the noisy image.

In [ ]:
rows, cols=size(im_noise)
Y=fft(im_noise);
heatmap(fftshift(log.(abs.(Y))),fill=cgrad(:grays,scale=:log), clims=(-2, 10))

In the plot of the Fourier transform above there are two white points near the centre of the image. These spots in the freqency domain show the noise - like the spikes in the 1D example above. The function below removes the two spots.

In [ ]:
#low pass filter
filterlow=fftshift(Y[:,:])
for y in 1:rows
    for x in 1:cols
        if (y-rows/2-2)^2+(x-cols/2-9)^2-2^2>=0 && (y-rows/2-1)^2+(x-cols/2+8)^2-2^2>=0
        else
            filterlow[x,y]=1e-3
        end
    end
end
heatmap((log.(abs.(filterlow))),fill=cgrad(:grays,scale=:log), clims=(-2, 10))

Next we can apply the inverse of the Fourier transform to get the original image back.

In [ ]:
X=ifft(filterlow)
Xn=Gray.(abs.(X))
Gray.(normHist(Xn))

More information about using Fourier Transforms for image processing can be found here: https://www.cs.unm.edu/~brayer/vision/fourier.html

# Edge Detection

Edge detection is another method of segmentation. It is used to highlight the edges of the image. Edges in images are defined where there are steep gradients. The gradient can be found by calculating the derivative. Since images are discrete arrays the derivative needs to be approximated. The most basic method is finite difference. If $Q$ is our original image and $P$ is our new image with edges detected the gradients can be approximated as,
\begin{equation}
P_{ij}=|2Q_{ij}-Q_{ij-1}-Q_{i-1j}|
\end{equation}
where $i$ and $j$ are pixels in the $x$ and $y$ directions, respectively. This function calculates the value of the new image by subtracting the values of the neighbouring pixels from the left and below. The larger the difference between the current pixel and the neighbouring pixels, the more likely this is to be an edge.

In [ ]:
camera=testimage("cameraman")

In [ ]:
# Edge detection method 1 - finite difference
newpic=zero(camera)
rows, cols = size(camera)
for x=2:rows-2
    for y=2:cols-2
        newpic[x,y]=abs(2*camera[x,y]-camera[x,y+1]-camera[x-1,y])
    end
end
newpic

Another method is applying a Sobel filter. This is similar to finite difference but uses a mask of 8 pixels weighted around a central point. In image processing this mask is known as a kernel and the process of applying the kernel to the image is a convolution. The Sobel filter uses two kernels, $\pmb{G}_x$ and $\pmb{G}_y$

\begin{equation}
\pmb{G}_x=\begin{bmatrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{bmatrix}\ \ \ \text{and}\ \ \ 
\pmb{G}_y=\begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \end{bmatrix}
\end{equation}

Note that $\pmb{G}_x$ is the transpose of $\pmb{G}_y$. These kernels allow the pixels on both the left and the right (or the one above and the one under) to be compared to the central pixel. The command ``imfilter`` can be used to apply the defined kernel.

In [ ]:
#sobel
sk=centered([1 0 -1; 2 0 -2; 1 0 -1]); #Define kernel, centered tells Julia the origin of the kernel is at the centre of the Array
sobel_x=imfilter(camera,sk); #Apply kernel in x direction
grad=imfilter(sobel_x, sk')  #Apply kernel in y direction

There is also an inbuild sobel kernel

In [ ]:
builtin=64*(imfilter(camera,Kernel.sobel())) #Mulitply by 64 to increase contrast

A more advance edge detection method is ``canny``. This combines the Gaussian filter with the Sobel edge detection kernel. This helps to remove the noise. More information can be found https://en.wikipedia.org/wiki/Canny_edge_detector and https://juliaimages.org/latest/function_reference/#Images.canny

In [ ]:
canny_edges=canny(camera, (Percentile(95), Percentile(40))) #Apply Canny filter, apply bounds to remove 'weak' edges
Gray.(canny_edges)

# Image Transforms

We can apply some transforms to images

In [ ]:
lake=testimage("lake_color") #Load image

In [ ]:
lake_rot=imrotate(lake,pi/2) #Rotate image

In [ ]:
lake_rot=lake' #Rotate image

In [ ]:
lake_rot=imrotate(lake,pi/4) #Rotate image

In [ ]:
lake_re=imresize(lake,ratio=1/2) #Resize image

In [ ]:
lake_c=lake[350:end-50,280:end-150] #Crop image

# Data Analysis
It is useful to be able to gain quantiative information from an image. To do this, we start by segmenting the image - separating out the objects of interest from the rest of the image. 

In [ ]:
rice=load("img/L8/rice.png") #Load image
rice=rice[1:end-100,:] #Crop image

In [ ]:
#Calculate and plot histogram
x_r, counts_r=imhist(rice);
plot(x_r, counts_r, xlims = (0,1), line=:stem, ylabel="Freqency",legend=false)

Two spikes can be seen in the histogram. The one on the left represents the background and the one of the right represents the grains of rice. So we can apply a threshold to segment the image

In [ ]:
thresh_r=rice .> 0.6 #apply a threshold
Gray.(thresh_r) #Display the resulting segmented image

```label_components``` allows us to count how many objects there are and also colour all the objects different colours.

In [ ]:
labels=label_components(thresh_r) #Labels each object separately
maximum(labels) #returns maximum label number, equal to number of object

In [ ]:
#Function to generate a random colour for each label
function get_random_color(seed)
    Random.seed!(seed)
    rand(RGB{N0f8})
end
#plot the multicoloured rice grains
plot(map(i->get_random_color(i), labels) )

Note that there are some partial grains around the edges and some odd pixels due to noise in the image 

In [ ]:
#Remove small rice grains - probably noise or partial grains
component_lengths(labels) #returns area for each label
labels_list=0:maximum(labels)
mask=labels_list[component_lengths(labels).>100] #Only keep rice grains with area greater than 100
new_labels=zeros(Int64,size(labels))
for j=1:length(mask)
    new_labels += ((labels.==mask[j]).*(j-1)) #Create new label array with only larger grains
end
@show maximum(new_labels)
plot(map(i->get_random_color(i), new_labels) )

Note that some of the  of the rice grains are joined together. They can be separated using [morphological operations](https://juliaimages.org/latest/function_reference/#ImageMorphology.dilate)  like ```erode``` and ```dilate```.

In [ ]:
#Use erode to separate grains. removes (remove 1 element around edge) - also can help remove noise
#Note that sometimes it is necessary to erode the image more than once
etr=erode(thresh_r)
Gray.(etr)

In [ ]:
#relabel separated rice grains
e_labels=label_components(etr)
plot(map(i->get_random_color(i), e_labels) )
maximum(e_labels)
Gray.(dilate(e_labels.>0))

In [ ]:
d_labels=zeros(Int64,size(e_labels))
for j=1:maximum(e_labels)
    d_labels=d_labels .+ (dilate(e_labels.==j))*(j-1) #dilate is opposite of erode, apply to each label separately
end
#replace overlaps with background colour to separate.
d_labels[d_labels.>maximum(e_labels)].=0 #Overlaps identified values over the maximum label calculated on eroded image
plot(map(i->get_random_color(i), d_labels))

Extension: can you find the smallest and largest grains of rice?

It is also possible to see how close the rice grains are to each other using ```distance_transform```

In [ ]:
#Find distance between grains of rice
F=feature_transform(d_labels.>0)#finding the closest "feature" for each location
D=distance_transform(F) #Compute the distance transform where each element represents a "feature" location. Specifically, D[i] is the distance between a location and a feature.
heatmap(D)

## Creating Art Work

Image processing can be used for both art and science. Above, we have shown how image processing can be use to gain quantitative data. Now we will use similar techniques to create artwork. I have chosen to create a piece of art based on Drawing hands by M. C. Escher: https://en.wikipedia.org/wiki/Drawing_Hands.

The aspects of the piece I aim to recreate are:
- Grayscale
- Outlined sleeves and realistic hands
- Hands in a circle

In [ ]:
arm=load("img/L8/Arm.JPG") #Load image
arm_c=Gray.(arm[:,30:end-30]) #Crop Image

In [ ]:
#Create and plot histogram
counts_a, x_a=imhist(arm_c); 
plot(counts_a, x_a, xlims = (0,1), line=:stem, ylabel="Freqency",legend=false)

In [ ]:
#Use histogram to choose threshold
arm_t=Gray.(arm_c).<0.58 #Foreground
back=Gray.(arm_c).>0.58 #Background
arm_seg=Gray.(arm_t.*arm_c) #Use thresholded foreground to separate arm from background
arm_seg[arm_seg .== 0].=1 #Set background color to white
arm_canny_edges=canny(Gray.(arm_c), (Percentile(99), Percentile(5))) #Find edges of arm
dil_arm=(dilate(dilate(arm_canny_edges))) #Dilate to make edge appear thicker, note we dilate twice
armedge=Gray.(1 .- dil_arm) #Create inverse of edge

In [ ]:
arm_outline=Gray.((1 .- arm_seg).*armedge) #Outline the inverse of the arm
arm_t[1:80,end-26:end].=0 #Black out corner of arm
arm_outback=arm_outline+back #Add white background to inverted arm
arm_f=[arm_outback[1:end-100,:];arm_seg[end-100:end,:]] #Add the outlined arm to segmented hand
#Take a subsection to make black so that image shows when overlayed
arm_sub=arm_f[:,end-26:end] 
arm_sub[arm_sub.==1].=0
arm_final=hcat(arm_f[:,1:end-26],arm_sub)
Gray.(arm_final)

In [ ]:
#Rotate thresholded arm
p2=imrotate(arm_t,pi)
p2[isnan.(p2).==1].=0 #remove nan values
m, n=size(arm_t)
l=hcat(arm_final,zeros(m+1,n-26)) #Add zeros to right increase size of image
r=hcat(zeros(m+2,n-26),p2) #Add zeros to left of rotated arm
left=(l .- r[2:end,2:end]) #Subtract thresholded arm from outlined arm
left[left.<=0].=0 #Remove some noise
Gray.(left)

In [ ]:
right=imrotate(left,pi) #rotate
full=left .+ right[2:end,2:end] #Add original and rotated images together to make circling arms
#Remove black stripe from centre
mid=full[:,181-26:182+1];
mid[mid.<=0.05].=1
#Combine images to create final piece
final=hcat(full[:,1:181-26],mid,full[:,182:end])
Gray.(final)

For more information on using Julia for image processing: 

- https://juliaimages.org/latest/quickstart/ 
- https://juliaimages.org/latest/function_reference/

Benchmark image collections are used by researchers to develop and evaluate their algorithms for image processing tasks. Here are a some of examples: 

- https://data.broadinstitute.org/bbbc/ 
- https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/

There are also many image datasets shared online that can be used for research or to appreciate as art
- https://earthobservatory.nasa.gov/images 
- https://www.nasa.gov/mission_pages/hubble/multimedia/index.html